In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d orion99/adience-age-gender-prediction-aligned-faces

In [ ]:
!unzip *.zip

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import pandas as pd
from tqdm import tqdm
import pickle
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
basepath = '/content/age-gender-data/'

#reading data from saved pickle file
def read_data(i):
  path = "/content/drive/My Drive/DL work/pickle/train_X_"+str(i)+".pickle"
  pickle_in = open(path,"rb")
  train_X = pickle.load(pickle_in)
  path = "/content/drive/My Drive/DL work/pickle/age_y/train_y_"+str(i)+".pickle"
  pickle_in = open(path,"rb")
  train_y = pickle.load(pickle_in)
  return train_X, train_y

#reading and resizing an image 
def read_and_resize(filepath):
    im = Image.open(filepath).convert('RGB')
    im = im.resize((227,227))
    im_array = np.array(im, dtype="uint8")
    #print(im_array.shape)
    im_array = np.array(im_array / 255.0, dtype="float32")
    return im_array

#preprocessing of dataframe (removing rows with incomplete info)
def filter_df(df):
  dict_age = {'(0, 2)' : 0,
                '(4, 6)' : 1,
                '(8, 12)' : 2,
                '(15, 20)' : 3,
                '(25, 32)' : 4,
                '(38, 43)' : 5,
                '(48, 53)' : 6,
                '(60, 100)' : 7}
  df['f'] = df.age.apply(lambda x: int(x in dict_age))
  df = df[df.f == 1]
  return df

def creating_data():
  dict_age = {'(0, 2)' : 0,
                '(4, 6)' : 1,
                '(8, 12)' : 2,
                '(15, 20)' : 3,
                '(25, 32)' : 4,
                '(38, 43)' : 5,
                '(48, 53)' : 6,
                '(60, 100)' : 7}
  for i in range(5):
    path = basepath+"fold_%i_data.txt"%i
    df = pd.read_csv(path, sep="\t")
    df = filter_df(df)
    train_X = []
    train_y = []

    for j in tqdm(range(df.shape[0])):
      try:
          img = basepath+"/aligned/"+str(df.at[j, 'user_id'])+"/landmark_aligned_face."+str(df.at[j, 'face_id'])+"."+str(df.at[j, 'original_image'])
          img_arr = read_and_resize(img)
          label = [0,0,0,0,0,0,0,0]
          label[dict_age[str(df.at[j, 'age'])]] = 1
          train_X.append(img_arr)
          train_y.append(label)
      except Exception:
        pass

    train_X = np.asarray(train_X)
    train_y = np.asarray(train_y)
    save_data(train_X, train_y, i)

#model definition
def build_model():
  model = keras.models.Sequential([
      keras.layers.Conv2D(96, (7,7), activation='relu', input_shape=(227,227,3)),
      keras.layers.MaxPooling2D((3,3), strides=2),
      keras.layers.Conv2D(256, (5,5), activation='relu'),
      keras.layers.MaxPooling2D((3,3), strides=2),
      keras.layers.Conv2D(384, (3,3), activation='relu'),
      keras.layers.MaxPooling2D((3,3), strides=2),
      keras.layers.Flatten(),
      keras.layers.Dense(512, activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(512, activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(8, activation='softmax')
  ])
  model.compile(loss="categorical_crossentropy", metrics=['acc'], optimizer="adam")
  model.summary()
  return model

#for saving data
def save_data(train_X, train_y, i):
  pickle_out = open("/content/drive/My Drive/DL work/pickle/train_X_%s.pickle"%i,"wb")
  pickle.dump(train_X, pickle_out)
  pickle_out.close()
  pickle_out = open("/content/drive/My Drive/DL work/pickle/age_y/train_y_%s.pickle"%i,"wb")
  pickle.dump(train_y, pickle_out)
  pickle_out.close()

#for opening image from dataframe
def open_img(row_i):
  path = basepath+"/aligned/"+str(df.at[row_i, 'user_id'])+"/landmark_aligned_face."+str(df.at[row_i, 'face_id'])+"."+str(df.at[row_i, 'original_image'])
  im = Image.open(path).convert('RGB')
  im = im.resize((227,227))
  print(im)


In [ ]:
#augmenting the image
datagen = ImageDataGenerator(
    rotation_range=6,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1)

def augment(im_array):
  im_array = datagen.random_transform(im_array)
  return im_array


In [ ]:
#prepare the dataset
creating_data()

100%|██████████| 3676/3676 [02:26<00:00, 25.10it/s]


In [ ]:
# predicting images
dict_age = {'(0, 2)' : 0,
                '(4, 6)' : 1,
                '(8, 12)' : 2,
                '(15, 20)' : 3,
                '(25, 32)' : 4,
                '(38, 43)' : 5,
                '(48, 53)' : 6,
                '(60, 100)' : 7}
def predict(path):
  im_array = read_and_resize(path)
  im_array = im_array.reshape(-1, 227, 227, 3)
  classes = model.predict(im_array)
  #print(classes[0])
  for k,v in dict_age.items():
    if v == np.argmax(classes):
      print(k)
      break

In [ ]:
#create the model
model = build_model()

In [ ]:
#loading the model
model = keras.models.load_model('/content/drive/My Drive/age.h5')

In [ ]:
#saving the model after training on each fold
model.save("/content/drive/My Drive/age.h5")

In [ ]:
#training the model on 5 different folds of data
for i in range(0,5):
  train_X, train_y = read_data(i)
  model.fit(train_X, train_y, validation_split = 0.10, epochs=20, verbose=1, batch_size=32, shuffle=True)
  train_X = np.array([augment(image) for image in train_X])
  model.fit(train_X, train_y, validation_split = 0.10, epochs=20, verbose=1, batch_size=32, shuffle=True)

In [ ]:
#evaluating performance of model on testing dataset
model.evaluate(train_X, train_y, verbose = 2)

In [ ]:
#for predicting on random images
for i in range(15):
  path = '/content/drive/My Drive/final year project/male face/'
  print(i+1, sep=' "" ', end='')
  predict(path+str(i+1)+'.jpg')